In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import catboost as cat
import optuna  # pip install optuna
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate

from joblib import dump, load

# Get Data

In [2]:
train = pd.read_csv("Data/train.csv", index_col = 0)
train.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f110,f111,f112,f113,f114,f115,f116,f117,f118,claim
id,,,,,,,,,,,,,,,,,,,,,
0,0.10859,0.004314,-37.566,0.017364,0.28915,-10.25100,135.12,168900.0,3.992400e+14,86.489,...,-12.2280,1.7482,1.90960,-7.11570,4378.80,1.2096,8.613400e+14,140.1,1.01770,1
1,0.10090,0.299610,11822.000,0.276500,0.45970,-0.83733,1721.90,119810.0,3.874100e+15,9953.600,...,-56.7580,4.1684,0.34808,4.14200,913.23,1.2464,7.575100e+15,1861.0,0.28359,0
2,0.17803,-0.006980,907.270,0.272140,0.45948,0.17327,2298.00,360650.0,1.224500e+13,15827.000,...,-5.7688,1.2042,0.26290,8.13120,45119.00,1.1764,3.218100e+14,3838.2,0.40690,1
3,0.15236,0.007259,780.100,0.025179,0.51947,7.49140,112.51,259490.0,7.781400e+13,-36.837,...,-34.8580,2.0694,0.79631,-16.33600,4952.40,1.1784,4.533000e+12,4889.1,0.51486,1
4,0.11623,0.502900,-109.150,0.297910,0.34490,-0.40932,2538.90,65332.0,1.907200e+15,144.120,...,-13.6410,1.5298,1.14640,-0.43124,3856.50,1.4830,-8.991300e+12,NaN,0.23049,1


In [3]:
X = train.drop("claim", axis = 1).copy()
y = train.claim.copy()

In [4]:
y.value_counts(normalize = True)

0    0.501508
1    0.498492
Name: claim, dtype: float64

In [5]:
test_data = pd.read_csv("Data/test.csv", index_col = 0)
sample_submission = pd.read_csv("Data/sample_solution.csv", index_col = 0)

# Train and test default LightGBM model

In [6]:
model = cat.CatBoostClassifier(
        loss_function="Logloss",
        eval_metric="AUC",
        task_type="GPU",
        random_seed=42
    ) 

cross_validation_score = cross_validate(model, X, y, scoring = 'roc_auc', cv = 5, n_jobs = 1, 
                                        return_train_score = True)#, return_estimator = True)

Learning rate set to 0.023395
0:	learn: 0.5451393	total: 76.3ms	remaining: 1m 16s
1:	learn: 0.5831696	total: 132ms	remaining: 1m 6s
2:	learn: 0.6102219	total: 180ms	remaining: 59.8s
3:	learn: 0.6378802	total: 229ms	remaining: 57s
4:	learn: 0.6579940	total: 278ms	remaining: 55.3s
5:	learn: 0.6757026	total: 330ms	remaining: 54.6s
6:	learn: 0.6851927	total: 378ms	remaining: 53.6s
7:	learn: 0.6939461	total: 428ms	remaining: 53s
8:	learn: 0.7075830	total: 479ms	remaining: 52.7s
9:	learn: 0.7171909	total: 530ms	remaining: 52.5s
10:	learn: 0.7276913	total: 577ms	remaining: 51.9s
11:	learn: 0.7333614	total: 630ms	remaining: 51.9s
12:	learn: 0.7386228	total: 675ms	remaining: 51.3s
13:	learn: 0.7468977	total: 725ms	remaining: 51.1s
14:	learn: 0.7526853	total: 770ms	remaining: 50.6s
15:	learn: 0.7591578	total: 820ms	remaining: 50.4s
16:	learn: 0.7627712	total: 873ms	remaining: 50.5s
17:	learn: 0.7680428	total: 921ms	remaining: 50.2s
18:	learn: 0.7719139	total: 969ms	remaining: 50s
19:	learn: 0.77

In [7]:
print("Mean AUC on training set:", np.mean(cross_validation_score['train_score']))
print("Mean AUC on test set:", np.mean(cross_validation_score['test_score']))

Mean AUC on training set: 0.8025785150071292
Mean AUC on test set: 0.7998693541439318


In [8]:
# Train on the whole training set and get predictions on the test set
model = cat.CatBoostClassifier(
        loss_function="Logloss",
        eval_metric="AUC",
        task_type="GPU",
        random_seed=42
    ) 
model.fit(X, y)
dump(model, 'Models/default_catboost_model.joblib') 

Learning rate set to 0.02311
0:	learn: 0.5450465	total: 55ms	remaining: 55s
1:	learn: 0.5828141	total: 129ms	remaining: 1m 4s
2:	learn: 0.6097153	total: 185ms	remaining: 1m 1s
3:	learn: 0.6376719	total: 244ms	remaining: 1m
4:	learn: 0.6578241	total: 301ms	remaining: 60s
5:	learn: 0.6723561	total: 369ms	remaining: 1m 1s
6:	learn: 0.6882419	total: 427ms	remaining: 1m
7:	learn: 0.6995759	total: 487ms	remaining: 1m
8:	learn: 0.7122173	total: 553ms	remaining: 1m
9:	learn: 0.7234676	total: 634ms	remaining: 1m 2s
10:	learn: 0.7333820	total: 690ms	remaining: 1m 2s
11:	learn: 0.7369136	total: 746ms	remaining: 1m 1s
12:	learn: 0.7450155	total: 805ms	remaining: 1m 1s
13:	learn: 0.7523359	total: 876ms	remaining: 1m 1s
14:	learn: 0.7576532	total: 936ms	remaining: 1m 1s
15:	learn: 0.7613619	total: 1s	remaining: 1m 1s
16:	learn: 0.7648613	total: 1.08s	remaining: 1m 2s
17:	learn: 0.7679999	total: 1.14s	remaining: 1m 2s
18:	learn: 0.7699841	total: 1.2s	remaining: 1m 1s
19:	learn: 0.7737823	total: 1.25s

['Models/default_catboost_model.joblib']

In [9]:
y_pred = model.predict_proba(test_data)
sample_submission.claim = y_pred[:, 1]
sample_submission.to_csv("Submissions/default_catboost_model.csv")

# Add Feature including number of nans per sample

In [ ]:
def feature_generation(data):

    # add feature with number of na's per row + standard deviation
    features = [col for col in data.columns if 'f' in col] # if 'f' is in the name
    data['count_na'] = data[features].isna().sum(axis=1)

    features += ['count_na']

    # now do imputation with mean
    data[features] = data[features].fillna(data[features].mean())

    return data

In [ ]:
X_w_nan = feature_generation(X)
test_data_w_nan = feature_generation(test_data)

In [ ]:
model = cat.CatBoostClassifier(
        loss_function="Logloss",
        eval_metric="AUC",
        task_type="GPU",
        random_seed=42
    ) 
model.fit(X_w_nan, y)
dump(model, 'Models/default_catboost_model_w_nan_count.joblib') 

In [ ]:
y_pred = model.predict_proba(test_data_w_nan)
sample_submission.claim = y_pred[:, 1]
sample_submission.to_csv("Submissions/default_catboost_model_w_nan_count.csv")

In [ ]:
feature_importance = pd.DataFrame(columns = ["Feature", "Importance"])
feature_importance.Feature = X_w_nan.columns
feature_importance.Importance = model.feature_importances_
feature_importance = feature_importance.sort_values("Importance", ascending = False)

_ = sns.barplot(data = feature_importance.iloc[:20], x = 'Importance', y = 'Feature')
plt.title("Top 20 most important features")

# Hyperparameter Tuning using Optuna

In [ ]:
from optuna.integration import LightGBMPruningCallback
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split


%%time
def objective(trial, Xy, ):
    train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.33, random_state=int(SEED), shuffle=True)
    train_pool = Pool(train_x, train_y)
    test_pool = Pool(test_x, test_y)
    
    # Parameters
    params = {
        'iterations' : trial.suggest_int('iterations', 50, 300),                         
        'depth' : trial.suggest_int('depth', 4, 10),                                       
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.3),               
        'random_strength' :trial.suggest_int('random_strength', 0, 100),                       
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
        'learning_rate' :trial.suggest_loguniform('learning_rate', 1e-3, 1e-1),
        'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter'])
    }
    # Learning
    model = cat.CatBoostClassifier(
        loss_function="Logloss",
        eval_metric="AUC",
        task_type="GPU",
        l2_leaf_reg=50,
        random_seed=42,
        border_count=64,
        **params
    )        
    model.fit(train_pool)
    # Predict
    preds = model.predict(test_pool)
    pred_labels = np.rint(preds)
    y_pred_boot = resample(pred_labels, n_samples = len(train_y))
    # Evaluation
    ROC_AUC_Score = roc_auc_score(train_y, y_pred_boot)
    print('ROC AUC Score of CatBoost =', ROC_AUC_Score)
    return ROC_AUC_Score

In [ ]:
%%time
study = optuna.create_study(direction = "maximize", sampler = TPESampler(seed=int(SEED)))
func = lambda trial: objective(trial, X_w_nan, y)
study.optimize(func, n_trials = 100, n_jobs = multiprocessing.cpu_count())

In [ ]:
print(f"\tBest value (AUC): {study.best_value:.5f}")
print(f"\tBest params:")

for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")

In [ ]:
param_grid = {
        "device_type": 'gpu',
        "n_estimators": 9590,
        "learning_rate": 0.18227838793671425,
        "num_leaves": 760,
        "max_depth": 5,
        "min_data_in_leaf": 700,
        "lambda_l1": 10,
        "lambda_l2": 25,
        "min_gain_to_split": 5.782744082905875,
        "bagging_fraction": 0.6,
        "bagging_freq": 1,
        "feature_fraction": 0.9
    }


In [ ]:
# Train model with best hyperparameters
model = lgbm.LGBMClassifier(objective="binary")
model.set_params(**study.best_params)
model.fit(X_w_nan, y)
dump(model, 'Models/default_lgbm_model_w_nan_count_tuned2.joblib') 

In [ ]:
y_pred = model.predict_proba(test_data_w_nan)
sample_submission.claim = y_pred[:, 1]
sample_submission.to_csv("Submissions/default_lgbm_model_w_nan_count_tuned2.csv")

In [ ]:
feature_importance = pd.DataFrame(columns = ["Feature", "Importance"])
feature_importance.Feature = X_w_nan.columns
feature_importance.Importance = model.feature_importances_
feature_importance = feature_importance.sort_values("Importance", ascending = False)

_ = sns.barplot(data = feature_importance.iloc[:20], x = 'Importance', y = 'Feature')
plt.title("Top 20 most important features")

In [ ]:
feature_importance = pd.DataFrame(columns = ["Feature", "Importance"])
feature_importance.Feature = X_w_nan.columns
feature_importance.Importance = model.feature_importances_
feature_importance = feature_importance.sort_values("Importance", ascending = False)

plt.figure(figsize=(10, 30))
_ = sns.barplot(data = feature_importance, x = 'Importance', y = 'Feature')
plt.title("Top 20 most important features")